In [1]:
import os
os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')

import myfuncs.myfuncs as mf
mf.read_directory()

import pandas as pd
pd.set_option('display.max_columns', None)

import re
import warnings
warnings.filterwarnings('ignore')

Your Current Directory is: C:\Users\jf79\Repos\REAP-Analytics
DataFrame: files
Number of records: 7 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,.git
1,.gitattributes
2,myfuncs
3,README.md
4,REAP_notebooks
5,REAP_venv
6,repofuncs


In [2]:
customer_columns = {
    'InteractionsKey':'interactionskey',
    'ETLTaskID':'etltaskid',
    'RowNumber':'rownumber',
    'GroupName':'groupname',
    'Type':'type',
    'Direction':'direction',
    'TimeQueued':'timequeued',
    'TimeEnded':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Username':'username',
    'UserID':'userid',
    'CustomerName':'customername',
    'CustomerContact':'customercontact',
    'QueueWait':'queuetime',
    'Duration':'duration',
    'TalkTime':'talktime',
    'CallHoldDuration':'holdtime',
    'WrapUpDuration':'wrapupduration',
    'AgentHandlingTime':'agenthandlingtime',
    'Result':'result',
    'ActivityCode':'activitycode',
    'AssistedBy':'assistedby',
    'TransferredTo':'transferredto',
    'MediaInteractionId':'interactionid',
    'Subject':'subject',
    'CaseReference':'casereference',
    'Tags':'tags',
    'Forwarded':'forwarded',
    'IPAddress':'ipaddress',
    'FormattedSource':'formattedsource',
    'ConversationID':'conversationid',
    'ConversationStartTime':'conversationstarttime',
    'ConversationEndTime':'conversationendtime',
    'ConversationDuration':'conversationduration'
}
master_columns = {
    'Interaction Id':'interactionid',
    'Type':'type',
    'Direction':'direction',
    'Time Queued':'timequeued',
    'Time Ended':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Customer Name':'customername',
    'Customer Contact':'customercontact',
    'Queue Time':'queuetime',
    'Duration':'duration',
    'Talk Time':'talktime',
    'Hold Time':'holdtime',
    'Wrap-up Duration':'wrapupduration',
    'Agent Handling Time':'agenthandlingtime',
    'Result':'result',
    'Activity Code':'activitycode',
    'Assisted By':'assistedby',
    'Transferred To':'transferredto',
    'Subject':'subject',
    'Conversation Start Time':'conversationstarttime',
    'Conversation End Time':'conversationendtime',
    'Conversation Duration':'conversationduration',
    'Agent Username':'agentusername',
    'Agent ID':'agentid',
    'Case Reference':'casereference',
    'Forwarded':'forwarded',
    'Source':'source',
    'Conversation ID':'conversationid',
    'IP Address':'ipaddress'
}

In [3]:
customer_service_data = mf.query_data(schema='Netcall', data='Interactions')

Successfully imported Interactions


In [4]:
path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)

mf.read_directory()

# Create an empty DataFrame to store the combined data
master_data = pd.DataFrame()
 
# Iterate over each folder in the root directory
for folder in os.listdir(path_to_data):
    folder_path = os.path.join(path_to_data, folder)
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate over each CSV file in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Ensure it's a CSV file
                file_path = os.path.join(folder_path, file)
 
                # Read CSV
                df = pd.read_csv(file_path)
 
                # Add a new column with the CSV filename
                df["Source_File"] = file
 
                # Append to master DataFrame
                master_data = pd.concat([master_data, df], ignore_index=True)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data
DataFrame: files
Number of records: 15 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,id_check.csv
1,Interactions Apr 2025 (excl Cust Serv)
2,Interactions August 2025 (excl Cust Serv)
3,Interactions Feb 2023 - Apr 2024 (excl Cust Serv)
4,Interactions Feb 2025 (excl Cust Serv)
5,Interactions Jan 2022 - Dec 2022 (excl Cust Serv)
6,Interactions Jan 2024 - Jun 2024 (excl Cust Serv)
7,Interactions July 2025 (excl Cust Serv)
8,Interactions Jun 24 - Jan 25 (excl Cust Serv)
9,Interactions Mar 2025 (excl Cust Serv)


In [5]:
customer_service = customer_service_data.copy()
customer_service.rename(
    columns=customer_columns,
    inplace=True
)
customer_service = customer_service[customer_service['groupname'] == 'Customer_Services']

columns_to_drop = [
    'interactionskey','etltaskid','rownumber',
    'username','userid','tags','formattedsource'
]
customer_service.drop(columns=columns_to_drop, inplace=True)
col = customer_service.pop('interactionid')
customer_service.insert(0, 'interactionid', col)
customer_service['groupname'] = customer_service['groupname'].str.replace('Customer_Services','Customer Services')

customer_service.sort_values(
    ['timequeued'],
    ascending=True,
    inplace=True
)

drop = customer_service[customer_service['interactionid'].duplicated(keep='first')]
customer_service.drop(
    drop.index,
    inplace=True
)
customer_service.reset_index(
    drop='index',
    inplace=True
)

customer_service['timequeued'] = pd.to_datetime(customer_service['timequeued'], format='mixed', dayfirst=True)
customer_service['timeended'] = pd.to_datetime(customer_service['timeended'], format='mixed', dayfirst=True)
customer_service['Year'] = customer_service['timequeued'].dt.year
customer_service['Month'] = customer_service['timequeued'].dt.month

In [6]:
master_df = master_data.copy()
master_df.rename(
    columns=master_columns,
    inplace=True
)

# Step 1: Split at the first dot (.)
master_df[['groupname', 'to_drop']] = master_df['Source_File'].str.split('.', expand=True)
 
# Step 2: Split at the first opening parenthesis (()
master_df[['groupname', 'to_drop']] = master_df['groupname'].str.split('(', expand=True)
 
# Step 3: Clean text (remove punctuation & normalize spaces)
master_df['groupname'] = master_df['groupname'].apply(lambda text: re.sub(r"[^\w\s]", "", text).strip())
master_df['groupname'] = master_df['groupname'].str.replace('Childrens', 'Children')
master_df['groupname'] = master_df['groupname'].str.replace('Children', 'Childrens')
master_df['groupname'] = master_df['groupname'].str.replace('ICAT6600', 'ICAT 6600')
master_df['groupname'] = master_df['groupname'].str.replace('Refugee', 'Refuge')
master_df['groupname'] = master_df['groupname'].str.replace('Refuge', 'Refugee')
master_df['groupname'] = master_df['groupname'].str.replace('Netcall HMS', 'HMS')
master_df['groupname'] = master_df['groupname'].str.replace('HMS', 'Netcall HMS')
master_df['groupname'] = master_df['groupname'].str.replace('Recovery Queue', 'Recovery')

col = master_df.pop('interactionid')
master_df.insert(0, 'interactionid', col)
col = master_df.pop('groupname')
master_df.insert(1, 'groupname', col)

master_df.sort_values(
    ['timequeued','agentid'],
    ascending=True,
    inplace=True
)

master_df.drop(
    columns=['Tags','Source_File','to_drop','Group'],
    inplace=True
)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(
    drop.index,
    inplace=True
)
master_df.reset_index(
    drop='index',
    inplace=True
)


master_df['timequeued'] = pd.to_datetime(master_df['timequeued'], format='mixed', dayfirst=True)
master_df['timeended'] = pd.to_datetime(master_df['timeended'], format='mixed', dayfirst=True)
master_df['Year'] = master_df['timequeued'].dt.year
master_df['Month'] = master_df['timequeued'].dt.month

In [7]:
final_df = pd.concat([master_df,customer_service])

values_to_change = [
    'Unknown','Completed','0'
]
for value in values_to_change:
    if value == '0':
        final_df['result'] = final_df['result'].str.replace('0','Failed')
    else:
        final_df['result'] = final_df['result'].str.replace(f'{value}','Other')

final_df['nullcount'] = final_df.isna().sum(axis=1)
final_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = final_df[final_df['interactionid'].duplicated(keep='first')]
final_df.drop(drop.index, inplace=True)
final_df.drop(columns='nullcount', inplace=True)

In [8]:
check = mf.id_check(final_df, 'interactionid', keep='first', directory=path_to_data)
mf.display(check)

if check is not None:
    final_df = final_df[~final_df['interactionid'].isin(check['interactionid'])]
    mf.display(final_df)

Exporting id_check to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data\id_check.csv

Successfully exported id_check to CSV
DataFrame: check
Number of records: 49,970 | Number of fields: 36
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customername,customercontact,queuetime,duration,talktime,holdtime,wrapupduration,agenthandlingtime,result,activitycode,assistedby,transferredto,subject,conversationduration,agentusername,agentid,casereference,forwarded,ipaddress,source,conversationid,conversationstarttime,Session ID,conversationendtime,Year,Month,MedSpecificID
1057060,6530313,Customer Services,Call,Inbound,2024-04-25 12:48:27,2024-04-25 14:15:22,Council Tax Support,Ctax Support Voice,Jacqueline Egbe,NaN,02070342090,00:36:43,00:50:11,00:50:11,00:00:00,00:01:01,00:51:12,Answered,CTAX - Moving In-Out,NaN,NaN,NaN,01:26:55,6421.0,6421,NaN,NaN,NaN,Dialogue,6190305.0,25/04/2024 12:48:27,NaN,25/04/2024 14:15:22,2024,4,NaN
1057111,6530491,Customer Services,Call,Inbound,2024-04-25 13:06:11,2024-04-25 13:45:23,Housing Benefits,Housing Benefits Voice,Joanne Davis,NaN,07474298929,00:31:19,00:07:54,00:07:54,00:00:00,00:00:01,00:07:55,Answered,NaN,NaN,NaN,NaN,00:39:12,7744.0,7744,NaN,NaN,NaN,Dialogue,6190483.0,25/04/2024 13:06:11,NaN,25/04/2024 13:45:23,2024,4,NaN
1057114,6530497,Customer Services,Call,Inbound,2024-04-25 13:06:44,2024-04-25 13:45:11,Housing Benefits,Housing Benefits Voice,Maureen Johnson,NaN,07541916378,00:31:24,00:07:02,00:07:02,00:00:00,00:00:43,00:07:45,Answered,DHP,NaN,NaN,NaN,00:38:27,21.0,021,NaN,NaN,NaN,Dialogue,6190489.0,25/04/2024 13:06:44,NaN,25/04/2024 13:45:11,2024,4,NaN
1057120,6530523,Customer Services,Call,Inbound,2024-04-25 13:08:54,2024-04-25 13:53:01,Council Tax and Housing Benefits Callback,Housing Benefits Voice,Joanne Davis,NaN,07841611900,00:36:36,00:07:31,00:07:31,00:00:00,00:02:00,00:09:31,Answered,NaN,NaN,NaN,NaN,00:44:07,7744.0,7744,NaN,NaN,NaN,Callback,6190515.0,25/04/2024 13:08:54,NaN,25/04/2024 13:53:01,2024,4,NaN
1057121,6530525,Customer Services,Call,Inbound,2024-04-25 13:09:08,2024-04-25 13:59:22,Council Tax Support,Ctax Support Voice,Maureen Johnson,NaN,07413677195,00:47:21,00:02:53,00:02:53,00:00:00,00:01:12,00:04:05,Answered,NaN,NaN,NaN,NaN,00:50:14,21.0,021,NaN,NaN,NaN,Dialogue,6190517.0,25/04/2024 13:09:08,NaN,25/04/2024 13:59:22,2024,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187650,6791695,Customer Services,Email,Inbound,2024-06-30 11:56:08,2024-07-01 10:25:33,Electoral Services,Electoral Services Email,Gillian Hoyle,NaN,pharmsforest@gmail.com,22:29:25,00:01:01,00:01:00,00:00:01,00:00:00,00:01:00,Replied,Postal - Proxy Enquiry,NaN,NaN,Ballot Mistake,22:29:25,4361.0,Unknown Agent ID,NaN,0.0,NaN,"To/CC contains ""Electoral.services@lbhf.gov.uk""",6444917.0,30/06/2024 11:56:08,NaN,01/07/2024 10:25:33,2024,6,NaN
1187668,6791731,Customer Services,Email,Inbound,2024-06-30 15:04:57,2024-07-01 10:26:08,Electoral Services,Electoral Services Email,Gillian Hoyle,NaN,victoria_heyworthdunne@hotmail.com,19:21:11,00:00:35,00:00:35,00:00:00,00:00:00,00:00:35,Replied,Registration Query,NaN,NaN,Poll card,19:21:11,4361.0,Unknown Agent ID,NaN,0.0,NaN,"To/CC contains ""Electoral.services@lbhf.gov.uk""",6444951.0,30/06/2024 15:04:57,NaN,01/07/2024 10:26:08,2024,6,NaN
1187691,6791777,Customer Services,Email,Inbound,2024-06-30 18:40:02,2024-07-03 16:36:52,FIS,FIS Email,Gillian Hoyle,NaN,solisbright2023@gmail.com,69:56:51,54:10:44,01:23:04,52:47:40,00:00:00,01:23:04,Other,FIS - Other,NaN,NaN,Housing,69:56:50,4361.0,Unknown Agent ID,NaN,1.0,NaN,"To/CC contains ""fis@lbhf.gov.uk""",6444995.0,30/06/2024 18:40:02,NaN,03/07/2024 16:36:52,2024,6,NaN
1187692,6791779,Customer Services,Email,Inbound,2024-06-30 20:09:49,2024-07-01 08:01:22,Cost of Living Crisis Email,Cost of Living Crisis Email,Umar Rajput,NaN,NoReply@lbhf.gov.uk,11:51:33,00:00:04,00:00:04,00:00:00,00:01:00,00:01:04,Other,Wrong Department,NaN,NaN,COL627045550 LBHF Crisis support advice,11:51:33,5764.0,5764,NaN,0.0,NaN,"To/CC contains ""costoflivingteam@lbhf.gov.uk""",6444997.0,30/06/2024 20:09:49,NaN

DataFrame: final_df
Number of records: 2,786,295 | Number of fields: 36
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customername,customercontact,queuetime,duration,talktime,holdtime,wrapupduration,agenthandlingtime,result,activitycode,assistedby,transferredto,subject,conversationduration,agentusername,agentid,casereference,forwarded,ipaddress,source,conversationid,conversationstarttime,Session ID,conversationendtime,Year,Month,MedSpecificID
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,Gbenga.Osunkoya@lbhf.gov.uk,85:46:29,00:04:05,00:04:05,00:00:00,00:00:04,00:04:09,Replied,Repair Status,NaN,NaN,RE: Inspection item Requires Attention - (BLK)...,85:46:28,4861,4861.0,NaN,0.0,NaN,Email,2516337.0,01/01/2022 02:15,NaN,04/01/2022 16:01,2022,1,NaN
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,NaN,Baboucarr.Gaye@lbhf.gov.uk,81:51:19,00:03:35,00:03:34,00:00:01,00:00:09,00:03:43,Replied,Repair Status,NaN,NaN,RE: Norland House Electrical Failure (Job Ref....,81:51:20,4861,4861.0,NaN,0.0,NaN,Email,2516345.0,01/01/2022 06:14,NaN,04/01/2022 16:05,2022,1,NaN
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,NaN,orbisreports@orbisprotect.com,78:56:42,00:03:06,00:03:06,00:00:00,00:00:16,00:03:22,Transferred,OOH - Related,NaN,Mehul Patel,[Orbis Protect] - Unallocated Cases,79:09:21,971,971.0,NaN,0.0,NaN,Email,2516347.0,01/01/2022 09:08,NaN,04/01/2022 16:17,2022,1,NaN
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,NaN,noreply@lbhf.gov.uk,78:41:00,00:14:11,00:14:10,00:00:01,00:01:00,00:15:10,Replied,Repair Status,NaN,NaN,Form submission from: Housing repair notificat...,78:41:00,971,971.0,NaN,0.0,NaN,Email,2516349.0,01/01/2022 09:38,NaN,04/01/2022 16:19,2022,1,NaN
4,2640689,Repair CSC,Email,Inbound,2022-01-01 11:49:00,2022-01-04 16:11:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,photobook@housemark.co.uk,76:21:50,00:05:42,00:05:42,00:00:00,00:00:03,00:05:45,Replied,Report New Repair,NaN,NaN,Inspection item Requires Attention - (BLK) 1-1...,76:21:50,4861,4861.0,NaN,0.0,NaN,Email,2516355.0,01/01/2022 11:49,NaN,04/01/2022 16:11,2022,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204814,8560871,Customer Services,Call,Outbound,2025-09-05 17:19:57,2025-09-05 17:20:38,Environment,Environment Voice,Darren Newman,,07592093803,N/A,00:00:34,00:00:34,00:00:00,00:00:10,00:00:44,Failed,,,,,00:00:41,NaN,NaN,,N/A,,NaN,8167955,05/09/2025 17:19:57,NaN,05/09/2025 17:20:38,2025,9,419549150
204815,8560921,Customer Services,Call,Inbound,2025-09-05 17:47:42,2025-09-05 17:49:36,Cost of Living Crisis,Cost of Living Crisis,Cush-Ashanti Porter,,07806753288,00:00:16,00:01:38,00:01:38,00:00:00,00:00:30,00:02:08,Answered,Cost of Living Payment Query,,,,00:01:54,NaN,NaN,,N/A,,NaN,8168003,05/09/2025 17:47:42,NaN,05/09/2025 17:49:36,2025,9,521566493
204816,8560925,Customer Services,Call,Inbound,2025-09-05 17:51:26,2025-09-05 17:55:12,Cost of Living Crisis,Cost of Living Crisis,Cush-Ashanti Porter,,07440155142,00:00:12,00:03:33,00:03:33,00:00:00,00:00:33,00:04:06,Answered,Food Bank Referral,,,,00:03:46,NaN,NaN,,N/A,,NaN,8168007,05/09/2025 17:51:26,NaN,05/09/2025 17:55:12,2025,9,454463479
204817,8560931,Customer Services,Email,Outbound,2025-09-05 17:55:13,2025-09-05 17:55:38,Cost of Living Crisis Email,Cost of Living Crisis Email,Cush-Ashanti Porter,,gbakiemrabrahamlincolnandrewib@gmail.com,N/A,00:00:24,00:00:24,00:00:00,00:01:01,00:01:25,Sent,Food Bank Referral,,,,00:00:25,NaN,NaN,,0,,NaN,8168013,05/09/2025 17:55:13,NaN,05/09/2025 17:55:38,2025,9,


In [9]:
mf.export_file(
    final_df, 
    directory = path_to_data, 
    df_name='Netcall Interactions Master Data',
    file_type='csv'
)

Exporting Netcall Interactions Master Data to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data\Netcall Interactions Master Data.csv

Successfully exported Netcall Interactions Master Data to CSV
